In [ ]:
#import statements

In [ ]:
class Tree():
    #initialize
    print("initialize tree")
    # Node root
    # make sure tree class is immutable

    def evaluate(x):
        #evaluates fitness of a tree
        #needs to be very efficient

        #scan/traverse tree for operators and look at left and right (its children)
        #incorporate order of operations
        #use if statements to determine next actions
        #recursive function to solve subtrees
        #ex: if + then add recursice simplification of one subtree to the simplification of other subtree
        
        #returns float or int
        val = 0
        return val

    def initialPop(numNodes):
        #start with one node
        #generate random operation
        #depending on operation determine children




class Node():
    #initialize
    # Node parent, left, right
    # Value: contents of the node (type: str)
    print("initialize node")


    